# Pytorch Custom Datasets


### 0. Importing PyTorch and setting up device-agnostic code

In [ ]:
import torch
from torch import nn

# Note: this notebook requires torch >= 1.10.0
torch.__version__

In [ ]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### 1. Get Data
- Our dataset is a subset of the Food101 dataset.
- Food101 starts with 101 different categories of food and 1000 images per class ( 750 training, 250 each testing)
- Our dataset starts with 3 classes of food and only 10% of the images (~75% training, 25 testing each)
- Why do this?
- When starting with ML project, it's important to try things on small scale and then increase the scale when necessary.
- The whole point is to speed up how fast you can experiment.

In [ ]:
import requests
import zipfile
from pathlib import Path

# Setup path to the data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# if the image folder doesn't exist , download it and perpare it
if image_path.is_dir():
    print(f"{image_path} directory already exists ..... skipping download")
else:
    image_path.mkdir(parents=True, exist_ok=True)

# Download the data
with open(data_path / "pizza_steak_sushi.zip", "wb")as f:
    #### Always copy the raw file from the github
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading.....")
    f.write(request.content)

# unzip the zip file downloaded
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unziping the dataset.......")
    zip_ref.extractall(image_path)

### 2. Becoming one with the data ( data preparation and data exploration )

In [ ]:
import os
def walk_through_dir(dir_path):
    """Walks through dir path returning its contents."""
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")

In [ ]:
walk_through_dir(image_path)

In [ ]:
# Setup training and testing paths **
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

#### 2.1 Visualizing and images

Lets write some code to:
1. Get all of the image paths
2. Pick a random image path using python's `random.choice()`
3. Get the image class name using `Pathlib.Path.parent.stem`
4. Since we are working with images, open the image using PIL library
5. We will show th image and print the data

In [ ]:
import random
from PIL import Image

# set Seed
random.seed(42)

# 1.  get all the image paths
image_path_lists = list(image_path.glob("*/*/*.jpg"))

# 2. Random image path and plot it
random_image_path = random.choice(image_path_lists)
#print(random_image_path)

# 3. Get the image class from the path name ( the image class is the name of the directory where the image is stored)
image_class = random_image_path.parent.stem
#print(image_class)

# 4. Open image 
img = Image.open(random_image_path)

# 5. print Metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image Height: {img.height}")
print(f"Image Width: {img.width}")
img


In [ ]:
# try to visulaize the image with mmatplotlib library
import numpy as np
import matplotlib.pyplot as plt

# Turn the image into an array
img_as_array = np.asarray(img)

# Plot the image with matplotlib
plt.figure(figsize=(10,7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image Shape: {img_as_array.shape} -> [Height, Width, Colour Channel]")
plt.axis(False);

### 3. Transforming Data
Before we can use our data with pytorch:
1. Turn your target data into tensors ( in our case, numerical representation of our images).
2. Turn it into a `torch.utils.data.Dataset`  sunsequently turned into `torch.utils.data.DataLoader`
3. 

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#### 3.1 Transforming data with torchvision.transforms
Transforms help you get your images ready to be used with a model/perform data augmentation Link: https://pytorch.org/vision/stable/transforms.html

In [ ]:
# Write a transform for an image
data_transform = transforms.Compose([
    # Resize our images to 64*64
    transforms.Resize(size=(64,64)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),
    # Turn the images into a torch.Tensor
    transforms.ToTensor()
])

In [ ]:
def plot_transformed_images(image_path, transform, n=3, seed=42):
    """
    Selects random images from a path of images and loads/Transforms
    them then plots the original vs the transformed version.
    """
    if seed:
        random.seed(42)
    random_image_path = random.sample(image_path, k=n)
    for image_path in random_image_path:
        with Image.open(image_path) as f:    
            fig, ax = plt.subplots(nrows=1, ncols=2)
            ax[0].imshow(f) # type: ignore
            ax[0].set_title(f"Original\nSize: {f.size}") # type: ignore
            ax[0].axis(False) # type: ignore

            # TRansform and plot target image
            transformed_image = transform(f).permute(1,2,0) # changing the order from (C,H,W) -> (H,W,C)
            ax[1].imshow(transformed_image)
            ax[1].set_title(f"Transformed\nShape: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

plot_transformed_images(image_path=image_path_lists,
                        transform=data_transform,
                        n=3,
                        seed=42
                        )

#### 4.1 Option 1. Loading image data using `ImageFolder`
We can load image classification data using `torchvision.datasets.ImageFolder` : https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html#torchvision.datasets.ImageFolder


In [ ]:
# Use the ImageFolder to create datasets(s)
# Pre-build datasets function developed ny Pytorch to load the data in certain structure format
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir, # type: ignore
                                  transform=data_transform, # A transform for the data (Custom ones)
                                  target_transform=None, # A transform for the target
                                  )

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform
                                 )
train_data, test_data

In [ ]:
# Get class names as list
class_names = train_data.classes
class_names

In [ ]:
# Get class names as dict()
class_dict = train_data.class_to_idx
class_dict

In [ ]:
# Check the length of our datasets
len(train_data), len(test_data)

In [ ]:
train_data.samples[0]

In [ ]:
# index on the train data Dataset to get a single image and label
img, label = train_data[0]
print(f"Image Tensor:\n {img}")
print(f"Image Shape: {img.shape}")
print(f"Image Datatype: {img.dtype}")
print(f"Image Label: {class_names[label]}")
print(f"Label Dataype: {type(label)}")

In [ ]:
# Rearrage the order of dimensions
img_permute = img.permute(1,2,0)

# print out different shapes
print(f"Original Shape: {img.shape} -> [Color_channels,Height,Width]")
print(f"Image PErmuted shape: {img_permute.shape} -> [Height, Width, color_channels]")

# Plot the image
plt.figure(figsize=(10,7))
plt.imshow(img_permute)
plt.axis("off")
plt.title(class_names[label], fontsize=14)

#### 4.1 Turn datasets into `Dataloader`
A `Dataloader` is going to help turn our datasets into iterables and we can customize the `batch_size` so our model can see batch size at one time.

In [ ]:
import os
os.cpu_count()

In [ ]:
# Turn traina and test into dataloader
from torch.utils.data import DataLoader
BATCH_SIZE = 1
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=1,#os.cpu_count(),
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             num_workers=1,#os.cpu_count(),
                             shuffle=True)

len(train_dataloader), len(test_dataloader)

In [ ]:
img, label = next(iter(train_dataloader))

# Batrch size will be one, you can change the batch size if you like
print(f"Image Shape: {img.shape} -> [batch_size, colour_channels, height, width]")
print(f"Label Shape: {label.shape}")

### 5 Option 2: Loading Image data with Custom `Dataset` instead using Pytorch nuild in functionality.
1. Want to be able to load images from file
2. Want to be able to get class names from the dataset
3. Want to be able to get classes as dictionary from the dataset

Pros:
* Can create a `Dataset` out of almost anything
* Not limited to pytorch pre-built `dataset` functions

Cons:
* Even though you could create `Datasets` out of almost anything, it doesnt mean it wil work.....
* Using a `Custom Dataset` often results in us writing more code, which could be prone to more errors or peformance issues.

All customs datasets in pytorch, often subclass - https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
 

In [ ]:
import os
import pathlib
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple, Dict, List

In [ ]:
# Instance of torchvision.datasets.ImageFolder()
train_data.classes, train_data.class_to_idx

#### 5.1 Creating a helper function to get class names
We want a function to:
1. Get the class names using `os.scandir` to traverse a target directory (ideally the directory is in standard image classification format)
2. Raise an error if te class names aren't found (if this happens, there might be something wrong with directort structure)
3. Turn the class names into a dict and  alist and return them.

In [ ]:
# Setup Path for target directory
from matplotlib import image


target_directory = train_dir
print(f"Target Dir: {target_directory}")

# Get the class names from the target directory
class_names_found = sorted([entry.name for entry in list(os.scandir(target_directory))])
class_names_found

In [ ]:
list(os.scandir(target_directory))

In [ ]:
# Turn the Above Steps into a executable function
def find_classes(directory: str) -> Tuple[list[str], Dict[str, int]]:
    """Finds the Class folder names in a target directory"""
    # 1. Get the class names by scanning the target directory
    classes = sorted(entry.name for entry in list(os.scandir(directory)) if entry.is_dir())

    # 2. Raise an error if class names not to be found
    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes in {directory}.... Please Check!!!")
    
    # 3. Create a dictionary of index labels ( comapre prefer numbers rather than strings as labels )
    class_to_idx = {class_name: i for i, class_name in enumerate(classes)}

    return classes, class_to_idx

#### 5.2 Create a custom `Dataset` to replicate `ImageFolder` functionality
* To create our Own dataset, we ewant to:
1. subclass `torch.utils.data.Dataset`
2. Init our subclass with a target dicrectory ( the directory we would like to get data from ) as well as a transform if we would like to transform our data
3. Create several attributes:
* paths - paths of out images
* transform - the transform we would like to perform
* classes - a list to get the target classes
* class_to_idx - a dict of the target classes mapped to integer labels
4. create a function to `load_images()`, this function will open images
5. Overwrite `__len()__` method to return the length of our dataset
Overwrite the `__getitem()__` method to returhn a given sample when passed an index

In [ ]:
# Write a custom dataset class
from torch.utils.data import Dataset

# 1. Subclass torch.utils.data.Dataset
class ImageFolderCustom(Dataset):
    # 2. Initialize our custom dataset
    def __init__(self, 
                targ_dir: str,
               transform=None):
        # 3. Create class attributes
        # Get all the images paths here
        self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg"))
        # Setup transforms
        self.transform = transform
        # Create classes adn class_to_idx
        self.classes, self.class_to_idx = find_classes(targ_dir)

    # 4. Create a function to load images
    def load_images(self, index: int) -> Image.Image:
        """Opens an image via path and returns it"""
        image_path = self.paths[index]
        return Image.open(image_path)
    
    # 5. Overwrite __len()__
    def __len__(self) -> int:
        """Returns the total number of samples"""
        return len(self.paths)
    
    # 6. Overwrite __getitem()__ method to return a sample
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        """Returns one sample of data, and the data label (X,y)"""
        img = self.load_images(index)
        class_name = self.paths[index].parent.name # Expects path in format: data_folder/class_name/image.jpg
        class_idx = self.class_to_idx[class_name]

        # Transform if necessary
        if self.transform:
            return self.transform(img), class_idx # Return data and lable index
        else:
            return img, class_idx # type: ignore # return untransformed image and label index

In [ ]:
# test our custom dataset function
train_transforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])


In [ ]:
# Test our custom ImageFolderCustom
train_data_custom = ImageFolderCustom(
    targ_dir=train_dir,
    transform=train_transforms
)

test_data_custom = ImageFolderCustom(
    targ_dir=test_dir,
    transform=test_transforms
)

train_data_custom, test_data_custom

In [ ]:
# The original one
train_data

In [ ]:
len(train_data), len(train_data_custom)

In [ ]:
len(test_data), len(test_data_custom)


In [ ]:
# Check for equality between original imageFolder Dataset and ImageFolderCustomDataset
print(train_data_custom.classes==train_data.classes)
print(test_data_custom.classes==test_data.classes)

#### 5.3 Create a function to display random images

1. Taken in `Dataset` and number of other paramerters such as class names and how many images to visualize.
2. To prevent the display getting out of hand, lets cap the number of images to see at 10.
3. Set the random seed for reproducibility.
4. Get a lsit of random sample indexes from the target dataset.
5. Setup a matlplotlib plot.
6. Loop through the random sample images and plot them with matpotlib.
7. Make sure the dimensions of our images line up with matplotlib.

In [ ]:
# 1. Create a function to take in a dataset
import random
def display_random_images(dataset: torch.utils.data.Dataset, # type: ignore
                          classes: List[str]=None, # type: ignore
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None): # type: ignore
    # 2. Adjust if n is very high
    if n > 10:
        n = 10
        display_shape = False
        print(f"For Display, Purpose, n shouldn't be larger than 10, setting to 10 and removing the display")
    
    # 3. Set the seed
    if seed:
        random.seed(seed)

    # 4. Get the random sample indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # 5. Setup Matplotlib plot
    plt.figure(figsize=(16,8))

    # 6. Loop through random indexes and plot them with matplotlib
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

        # 7. Adjust tensor dimensions for plotting
        targ_image_adjust = targ_image.permute(1,2,0) # Changing [C,H,W] -> [H,W,C]

        # Plot the adjusted Image
        plt.subplot(1,n,i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        if classes:
            title = f"Class: {classes[targ_label]}"
            if display_shape:
                title = title + f"\nShape: {targ_image_adjust.shape}"
        plt.title(title)

In [ ]:
# Display random images fromt the ImageFolder created in ImageFolder
display_random_images(dataset=train_data,
                      n=5,
                      classes = class_names,
                      seed=None)

In [ ]:
# Display random images from the ImageFolderCustom Dataset
display_random_images(dataset=train_data_custom,
                      n=5,
                      classes=class_names,
                      )

#### 5.4 Turn custom Loaded images into DataLoaders

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE = 32
NUMBER_WORKERS = 0  # os.cpu_count()
train_dataloader_custom = DataLoader(dataset=train_data_custom,
                                     batch_size=BATCH_SIZE,
                                     num_workers=NUMBER_WORKERS,
                                     shuffle=True)

test_dataloader_custom = DataLoader(dataset=test_data_custom,
                                    batch_size=BATCH_SIZE,
                                    num_workers=NUMBER_WORKERS,
                                    shuffle=False)
train_dataloader_custom, test_dataloader_custom

In [ ]:
# Get Image adn label from custom Dataloader
img_custom, label_custom = next(iter(train_dataloader_custom))

# Print out the shapes
img_custom.shape, label_custom.shape

### 6 Other forms of transforms ( Data Augmentation )
* Data augmentation is the process of artificially addign diversity to your trainign data.
* In this case of image data, this may mean applying various image transformation to the trainig images.
* Lets take a look at one aprticular type of data augmentation used to train PyTorch vision models to state of the art levels....
* this practise enables model to generalize more on the unseeen data
* blog post: https://pytorch.org/blog/how-to-train-state-of-the-art-models-using-torchvision-latest-primitives/
* Augmentations : https://pytorch.org/vision/main/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py

In [ ]:
# lets look at Trivial Augment -> https://pytorch.org/vision/main/generated/torchvision.transforms.TrivialAugmentWide.html#torchvision.transforms.TrivialAugmentWide

from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])



In [ ]:
image_path

In [ ]:
# Get all the image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))
image_path_list[:10]

In [ ]:
# Plot random transformed images
plot_transformed_images(image_path=image_path_list,
                        transform=train_transform,
                        n=4,
                        seed=42)

### 7 Model 0: TinyVGG without the data augmentation

#### 7.1 Creating Transform and loading data for Model 0

In [ ]:
# Create simple transform
simple_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [ ]:
# 1. Load and transform data
from torchvision import datasets

train_data_simple = datasets.ImageFolder(root=train_dir,
                                         transform=simple_transform)
test_data_simple = datasets.ImageFolder(root=test_dir,
                                        transform=simple_transform)

# 2, Turn the datasets into DataLoaders
import os
from torch.utils.data import DataLoader

# Setup batch size and the number of works
BATCH_SIZE = 32
NUM_WORKERS = 1#os.cpu_count()

# CCreate DataLaoders
train_dataloader_simple = DataLoader(dataset=train_data_simple,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(dataset=test_data_simple,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS)

#### 7.2 Create TinyVGG model class

reading article : https://horace.io/brrr_intro.html

In [ ]:
class TinyVGG(nn.Module):
    """Model Architecture copying TinyVGG from cnn Explainer"""
    def __init__(self, input_shape: int,
                hidden_units: int,
                output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default value of stride is same as kernel size

        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )

        self.classifier_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13,
                      out_features=output_shape)
        )


    def forward(self, x):
        # x = self.conv_block_1(x)
        # #print(x.shape)
        # x = self.conv_block_2(x)
        # #print(x.shape)
        # x = self.classifier_layer(x)
        #return x
        return self.classifier_layer(self.conv_block_2(self.conv_block_1(x))) # benefits of using this type of syntax

In [ ]:
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3,
                  hidden_units=10,
                  output_shape=len(class_names)).to(device)

In [ ]:
model_0

#### 7.3 Try a forward pass on a single image data ( to test te model)

In [ ]:
# Get a single image batch
image_batch, label_batch = next(iter(train_dataloader_simple))
image_batch.shape, label_batch.shape

In [ ]:
# Try a forward pass
model_0(image_batch.to(device))

##### To get the output shapes of the layers created int he model class

In [ ]:
# Defined functions to calcluate the output shapes from every layer
import math

def num2tuple(num):
    return num if isinstance(num, tuple) else (num, num)

def conv2d_output_shape(h_w, hiddle_shape: int=10, batch_size: int=32, kernel_size=1, stride=1, pad=0, dilation=1):
    h_w, kernel_size, stride, pad, dilation = num2tuple(h_w), \
        num2tuple(kernel_size), num2tuple(stride), num2tuple(pad), num2tuple(dilation)
    pad = num2tuple(pad[0]), num2tuple(pad[1])
    
    h = math.floor((h_w[0] + sum(pad[0]) - dilation[0]*(kernel_size[0]-1) - 1) / stride[0] + 1)
    w = math.floor((h_w[1] + sum(pad[1]) - dilation[1]*(kernel_size[1]-1) - 1) / stride[1] + 1)
    
    return batch_size,hiddle_shape,h, w

def convtransp2d_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1, out_pad=0):
    h_w, kernel_size, stride, pad, dilation, out_pad = num2tuple(h_w), \
        num2tuple(kernel_size), num2tuple(stride), num2tuple(pad), num2tuple(dilation), num2tuple(out_pad)
    pad = num2tuple(pad[0]), num2tuple(pad[1])
    
    h = (h_w[0] - 1)*stride[0] - sum(pad[0]) + dilation[0]*(kernel_size[0]-1) + out_pad[0] + 1
    w = (h_w[1] - 1)*stride[1] - sum(pad[1]) + dilation[1]*(kernel_size[1]-1) + out_pad[1] + 1
    
    return h, w

def conv2d_get_padding(h_w_in, h_w_out, kernel_size=1, stride=1, dilation=1):
    h_w_in, h_w_out, kernel_size, stride, dilation = num2tuple(h_w_in), num2tuple(h_w_out), \
        num2tuple(kernel_size), num2tuple(stride), num2tuple(dilation)
    
    p_h = ((h_w_out[0] - 1)*stride[0] - h_w_in[0] + dilation[0]*(kernel_size[0]-1) + 1)
    p_w = ((h_w_out[1] - 1)*stride[1] - h_w_in[1] + dilation[1]*(kernel_size[1]-1) + 1)
    
    return (math.floor(p_h/2), math.ceil(p_h/2)), (math.floor(p_w/2), math.ceil(p_w/2))

def convtransp2d_get_padding(h_w_in, h_w_out, kernel_size=1, stride=1, dilation=1, out_pad=0):
    h_w_in, h_w_out, kernel_size, stride, dilation, out_pad = num2tuple(h_w_in), num2tuple(h_w_out), \
        num2tuple(kernel_size), num2tuple(stride), num2tuple(dilation), num2tuple(out_pad)
        
    p_h = -(h_w_out[0] - 1 - out_pad[0] - dialation[0]*(kernel_size[0]-1) - (h_w[0] - 1)*stride[0]) / 2
    p_w = -(h_w_out[1] - 1 - out_pad[1] - dialation[1]*(kernel_size[1]-1) - (h_w[1] - 1)*stride[1]) / 2
    
    return (math.floor(p_h/2), math.ceil(p_h/2)), (math.floor(p_w/2), math.ceil(p_w/2))

#### 7.4 Use `torchinfo` to get an idea of the shapes going through out model

In [ ]:
## Another way to get the informatin of our model ####
from torchinfo import summary
#model = TinyVGG(input_shape=3,hidden_units=10,output_shape=10)
summary(model_0,input_size=(BATCH_SIZE,3,64,64))

#### 7.5 Crete a train dna test step loop functions

* `train_step()`: takes in a mdoel and dataloader and trains the model on the dataloader
* `test_step()`: takes in a model and the dataloader and evaluates the model on the dataloader


In [ ]:
# Create train_step funcxtion
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device=device):
    # Put the model to train mode
    model.train()

    # Setup train loss and train accuracy
    train_loss, train_acc = 0, 0

    # loop through dataloader data batches
    for batch, (X,y) in enumerate(dataloader):
        # send data tot eh target device
        X, y = X.to(device), y.to(device)

        # 1. forward Pass
        y_pred = model(X) # output is in logits

        # 2. Calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Soptimizer to zero grad
        optimizer.zero_grad()

        # 4. loss backward
        loss.backward()

        # 5. Optmizer step
        optimizer.step()

        # Calculate the accuracy metric
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class==y).sum().item()/len(y_pred)

    # Adjust mertrics to get the average metrics per batch
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    return train_loss, train_acc

In [ ]:
# Create a test step

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):
    # Put the model to eval mode
    model.eval()

    # setup test loss and test accuract metrics
    test_loss, test_acc = 0, 0

    # Trurn on inference mode
    with torch.inference_mode():
        # loop througfh thte dataloader batches
        for batch, (X,y) in enumerate(dataloader):
            # convert to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate the loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate the accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels==y).sum().item()/len(test_pred_labels))

    # Adjust the metrics to get the average of the accuracy per batch
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    return test_loss, test_acc

#### 7.6 Creating a `train()` function to combine `train_step()` and `test_step()`

In [ ]:
from tqdm.auto import tqdm

# 1. Create a train function that takes in various model parameters + optimizer + dataloader +++
def train_loop(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          device=device
          ):
    # 2. Create empty results dictionary
    results = {"train_loss":[],
               "train_acc":[],
               "test_loss":[],
               "test_acc":[]
               }
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        # 4. Print out  whats happening
        print(f"Epoch: {epoch} | Train Loss: {train_loss: .4f} | Train acc: {train_acc: .4f} | Test loss: {test_loss: .4f} | Test acc: {test_acc: .4f}")

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. return the filled results at the end of the epochs
    return results

#### 7.7 Train and evaluate our model

In [ ]:
device="cuda"
# set the random seed
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# set the number of epochs
NUM_EPOCHS = 30

# recreate an instance of tinyVGG
model_0 = TinyVGG(input_shape=3, # Number of color channels of our target images
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)

# Setup a loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=0.001)

# start the timer 
from timeit import default_timer as timer
start_time = timer()

# train the model_0
model_0_results = train_loop(model=model_0, # type: ignore
                             train_dataloader=train_dataloader_simple,
                             test_dataloader=test_dataloader_simple,
                             optimizer=optimizer,
                             loss_fn=loss_fn,
                             epochs=NUM_EPOCHS,
                             device=device)

end_time = timer()
print(f"Total trainig timing for {NUM_EPOCHS} : {end_time - start_time: .3f} seconds")


In [ ]:
model_0_results

#### 7.8 plot the loss curve of model 0
* A **loss curve** is a way to track your models progress over time.

In [ ]:
# get the model_0_results keys
model_0_results.keys()

In [ ]:
from typing import Dict

def plot_loss_curves(results: Dict[str,list[float]]):
    """Plot training curves of a reuslt dictionary"""
    # get the loss values of the results dicitonary
    loss = results["train_loss"]
    test_loss = results["test_loss"]

    # Get the accuracy values of the results duitionary
    accuracy = results["train_acc"]
    test_accuracy = results["test_acc"]

    # Figure out how many epochs were there
    epochs = range(len(results["train_loss"]))

    # setup the plott
    plt.figure(figsize=(15,7))

    # plot the loss
    plt.subplot(1,2,1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    # plot the accuracy
    plt.subplot(1,2,2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend();


In [ ]:
plot_loss_curves(results=model_0_results)

### 8 What should an ideal loss curve look like?
* Interpreting loss curves: https://developers.google.com/machine-learning/testing-debugging/metrics/interpretic
* pre-trained models: https://pytorch.org/vision/0.8/models.html

### 9 Model 1: TinyVGG with Data augmentation
* Now lets try another modeling experiment this time using the same model as before with some data augmentation

#### 9.1 Create transform with data augmentation

In [ ]:
import torch
from torch import nn
from torchvision import transforms

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# create training transform with TrivialAugment
train_transform_trivial = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

test_transform_trivial = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

#### 9.2 Create train andtest `Dataset` and `DataLoaders` with Data Augmentation

In [ ]:
from torchvision import datasets
from pathlib import Path
# Setup path to the data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

In [ ]:
train_dir = image_path / "train"
test_dir = image_path / "test"

In [ ]:
# Turn our Image folder to datasets
train_data_augment = datasets.ImageFolder(root=train_dir,
                                          transform=train_transform_trivial,
                                          )
test_data_simple = datasets.ImageFolder(root=test_dir,
                                        transform=test_transform_trivial)

In [ ]:
# Turn datasets into DataLoaders
import os
from torch.utils.data import DataLoader
BATCH_SIZE = 32
NUM_WORKERS = 1 #os.cpu_count()

torch.manual_seed(42)
train_dataloader_augmented = DataLoader(dataset=train_data_augment,
                                        shuffle=True,
                                        batch_size=BATCH_SIZE,
                                        num_workers=NUM_WORKERS)
test_dataloader_simple = DataLoader(dataset=test_data_simple,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS)

#### 9.3 Create another model_1 and train it
This time we will be using the same model class but with updated data augmentation.

In [ ]:
# Create model_1 and send it to the target device
torch.manual_seed(42)

model_1 = TinyVGG(input_shape=3,
                  hidden_units=10,
                  output_shape=len(train_data_augment.classes)).to(device)

model_1


Now lets create our loss function and an optimizer and call upon our `train()` function to train and evaluate our model.

In [ ]:
# Set random seed
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set the number of epochs
NUM_EPOCHS = 30

# setup loss and optimizer function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_1.parameters(),
                             lr=0.001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train the model 1
model_1_results = train_loop(model=model_1,
                        train_dataloader = train_dataloader_augmented,
                        test_dataloader = test_dataloader_simple,
                        optimizer=optimizer,
                        loss_fn= loss_fn,
                        epochs=NUM_EPOCHS,
                        device=device)

# End the timer and print out the time taken
end_time = timer()
print(f"Total training time for model_1: {end_time-start_time:.3f}")

#### 9.4 Plot loss curvers of  model_1

In [ ]:
# Importing libraries
from typing import Dict
import matplotlib.pyplot as plt

In [ ]:
plot_loss_curves(model_1_results)

### 10 Comparing our model results
Afetr evaluating oour models experiments on their own, its important to compare then to each other

There's few different ways to do this:
1. Hard Coding ( What we are doing)
2. Pytorch + tensorboard
3. Weights and Biases
4. MLFLow

In [ ]:
import pandas as pd
model_0_df = pd.DataFrame(model_0_results)
model_1_df = pd.DataFrame(model_1_results)
model_0_df


In [ ]:
# setup a plot
plt.figure(figsize=(15,10))

# Get number of epochs
epochs = range(len(model_0_df))

# plot the trainloss
plt.subplot(2,2,1)
plt.plot(epochs, model_0_df["train_loss"], label="Model 0")
plt.plot(epochs, model_1_df["train_loss"], label="Model 1")
plt.title("Train Loss")
plt.xlabel("Epochs")
plt.legend()

# plot the Test loss
plt.subplot(2,2,2)
plt.plot(epochs, model_0_df["test_loss"], label="Model 0")
plt.plot(epochs, model_1_df["test_loss"], label="Model 1")
plt.title("Test_loss")
plt.xlabel("Epochs")
plt.legend()

# plot the Train Accuracy
plt.subplot(2,2,3)
plt.plot(epochs, model_0_df["train_acc"], label="Model 0")
plt.plot(epochs, model_1_df["train_acc"], label="Model 1")
plt.title("Train Accuracy")
plt.xlabel("Epochs")
plt.legend()

# plot the Test Accuracy
plt.subplot(2,2,4)
plt.plot(epochs, model_0_df["test_acc"], label="Model 0")
plt.plot(epochs, model_1_df["test_acc"], label="Model 1")
plt.title("Test Accuracy")
plt.xlabel("Epochs")
plt.legend()

### 11 Making a prediction on a custom image
Although we have trained our model on custom data ..... how do you make a prediction on a sample thats not in train and also in test dataset

In [ ]:
# Download custom image
import requests

# setup custom image path
custom_image_path = data_path / "steak.jpeg"

# Download the image if it doesn not exist
if not custom_image_path.is_file():
    with open(custom_image_path, 'wb') as f:
        # When downloadign the image
        request = requests.get("https://therecipecritic.com/wp-content/uploads/2019/05/besthomemadepizzahero.jpg")
        print(f"Downloading..... {custom_image_path}")
        #print(request)
        f.write(request.content)
else:
    print(f"{custom_image_path} already exists, skipping downloading")

In [ ]:
custom_image_path = data_path / "steak.jpeg"

#### 11.1 Loading custom image with Pytorch
we ahve to make our custom image is in the same format as the data our model was trained on.
* In tensor form with datatype(torch.float32)
* of shape 64x64x3
* On the right Device
We can read an image into Pytorch using: https://pytorch.org/vision/stable/generated/torchvision.io.read_image.html#torchvision.io.read_image

In [ ]:
import torchvision

# read the custom image
custom_image_uint8 = torchvision.io.read_image(str(custom_image_path))

In [ ]:
custom_image_uint8

In [ ]:
plt.imshow(custom_image_uint8.permute(1,2,0))

In [ ]:
print(f"Custom image tensor:\n {custom_image_uint8}")
print(f"Custom image shape: {custom_image_uint8.shape}")
print(f"Custom Image datatypes: {custom_image_uint8.dtype}")

#### 11.2 Making predictions on a custom image with a trained pytorch model

In [ ]:
# Error: Wrong image datatype
model_1.eval()
with torch.inference_mode():
    model_1(custom_image_uint8.to(device))

In [ ]:
# Load in the custom image and convert to torch.float32
custom_image = torchvision.io.read_image(path=str(custom_image_path)).type(torch.float32) / 255.
custom_image

In [ ]:
plt.imshow(custom_image.permute(1,2,0))

In [ ]:
# Error cause image not in correct size
model_1.eval()
with torch.inference_mode():
    model_1(custom_image.to(device))

In [ ]:
# create a transform pipeline to resize the image
custom_image_transform = transforms.Compose([
    transforms.Resize(size=(64,64))
])

# Transfor the custom image
custom_image_transformed = custom_image_transform(custom_image)

# print out he shapes
print(f"Original image shape: {custom_image.shape}")
print(f"Transformed Image shape: {custom_image_transformed.shape}")

In [ ]:
plt.imshow(custom_image_transformed.permute(1,2,0))

In [ ]:
# this error we forgot to add batch dimension to it
model_1.eval()
with torch.inference_mode():
    model_1(custom_image_transformed.to(device))

In [ ]:
# Adding an batch  dimension to the image
custom_image_transformed.shape, custom_image_transformed.unsqueeze(0).shape

In [ ]:
#
model_1.eval()
with torch.inference_mode():
    custom_image_pred = model_1(custom_image_transformed.unsqueeze(0).to(device))

In [ ]:
custom_image_pred

**Note** : To make the predictions on a custom image we had to :
* Load the image and turn it into a tensor
* Make sure the image was on the same datatype as the model
* Make sur ethe image was of the same shape as the data the mmodel was trained on 
* Make sure the image was on the same device as the model

In [ ]:
# Convert logits -> prediction probabilities
custom_image_pred_probs = torch.softmax(custom_image_pred, dim=1)
custom_image_pred_probs

In [ ]:
# Convert prediction probabilities -> predicted labels
custom_image_pred_labels = torch.argmax(custom_image_pred_probs, dim=1)

In [ ]:
train_data_augment.classes[custom_image_pred_labels]

#### 11.3 Putting custom image predictions together: building a function
Ideal Outcome:
* A function where we pass an image path to and have our model predict on that image
and plot the image + prediciton.

In [ ]:
from typing import List
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        class_names: List[str]=None,
                        transform=None,
                        device=device):
    """Makes a prediction on target image with a trained model and plots the image and prediction"""
    # load the image
    target_image = torchvision.io.read_image(str(image_path)).type(torch.float32)

    # Divide the image pixel values by 255 to get them between 0-1
    target_image = target_image / 255.

    # transform the data if necessary
    if transform:
        target_image = transform(target_image)

    # Turn on eval model and make the prediction
    model.eval()
    with torch.inference_mode():
        # Add an extra dimension to the image ( this is the batch dimension)
        target_image = target_image.unsqueeze(dim=0)

        # Make a prediciton on the image with an extra dimension
        target_image_pred = model(target_image.to(device))
        
    # Convert logits -> prediction probabilities
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # Convert the prediction proabbilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # Plot the image alongside the prediciton and prediciton probabilities
    plt.imshow(target_image.squeeze().permute(1,2,0)) # remove the batch dimension and colour channel to last [H,W,C]
    if class_names:
        title = f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu(): .3f}"
    else:
        title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu(): .3f}"
    plt.title(title)
    plt.axis(False)


In [ ]:
# setup custom image path
custom_image_path = data_path / "steak.jpeg"
pred_and_plot_image(model=model_1,
                    image_path=str(custom_image_path),
                    class_names=train_data_augment.classes,
                    transform=custom_image_transform,
                    device=device)